In [1]:
import os
import sys

import pandas as pd
import numpy as np
import geopandas as gpd
import folium

import matplotlib.pyplot as plt
import seaborn as sns

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  set_use_pygeos()
/tmp/ipykernel_35118/4027295472.py:6: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


2024-01-16 11:26:22 Configured OSMnx 1.1.2
2024-01-16 11:26:22 HTTP response caching is on


In [2]:
gdf_mun = aup.gdf_from_db('hexgrid_8_city_2020', 'hexgrid')

In [3]:
print(gdf_mun.shape)
gdf_mun.head(2)

(317818, 6)


,hex_id_8,geometry,CVEGEO,NOMGEO,city,type
0,886d20c567fffff,"POLYGON ((-93.06006 17.95976, -93.05563 17.962...",27004,Centro,Villahermosa,urban
1,886d209a51fffff,"POLYGON ((-92.73815 18.23927, -92.73372 18.241...",27004,Centro,Villahermosa,rural


In [4]:
table = 'nearshoring_denue_23_hex'
schema = 'public_interest'
query = f"SELECT * FROM {schema}.{table} WHERE \"res\" = 8"
near_gdf = aup.gdf_from_query(query)
print(near_gdf.shape)
near_gdf.head(2)

(4108, 9)


,hex_id,geometry,nearshoring_industry,per_ocu_num_near_sum,denue,per_ocu_num_tot_sum,nearshoring_industry_pct,nearshoring_workforce_pct,res
0,88498eae3bfffff,"POLYGON ((-102.29558 21.73638, -102.29099 21.7...",2,95,2,95,1.0,1.0,8
1,884988d95bfffff,"POLYGON ((-102.37130 21.86355, -102.36671 21.8...",1,325,1,325,1.0,1.0,8


## Nearshore industries by city

In [5]:
near_mun = gdf_mun.merge(near_gdf.drop(columns=['geometry']), right_on='hex_id',
                         left_on='hex_id_8')
print(near_mun.shape)
near_mun.head(2)

(3266, 14)


,hex_id_8,geometry,CVEGEO,NOMGEO,city,type,hex_id,nearshoring_industry,per_ocu_num_near_sum,denue,per_ocu_num_tot_sum,nearshoring_industry_pct,nearshoring_workforce_pct,res
0,886d20892dfffff,"POLYGON ((-92.90866 18.01244, -92.90423 18.015...",27004,Centro,Villahermosa,urban,886d20892dfffff,1,3,2,11,0.5,0.272727,8
1,886d20c6e3fffff,"POLYGON ((-92.92568 17.96637, -92.92125 17.969...",27004,Centro,Villahermosa,urban,886d20c6e3fffff,1,3,5,20,0.2,0.150000,8


## Top cities

In [6]:
top_10 = near_mun.groupby('city').sum(numeric_only=True).sort_values(by='nearshoring_industry', ascending=False).head(11)
top_10 = top_10.add_prefix('23_')
top_10 = top_10.reset_index()
top_10

,city,23_nearshoring_industry,23_per_ocu_num_near_sum,23_denue,23_per_ocu_num_tot_sum,23_nearshoring_industry_pct,23_nearshoring_workforce_pct,23_res
0,CDMX,1075,43929,20018,257146,35.765570,71.098477,3160
1,Guadalajara,546,35854,9379,176147,31.639101,50.700437,1960
2,ZMVM,536,30237,14081,208865,30.596413,50.473633,2800
3,Monterrey,473,49819,5970,204899,60.360045,83.144549,2208
4,Tijuana,301,37714,2086,119462,23.161053,38.973425,872
5,Queretaro,269,33968,2356,99178,26.571924,37.562483,888
6,Puebla,237,18292,3976,66202,16.471224,28.196277,1000
7,Juarez,233,51744,1115,97497,23.188630,43.724470,704
8,Toluca,228,17442,3073,70483,14.055885,22.981951,824
9,Saltillo,163,29441,1202,59764,29.623333,40.237237,696


## Historical data

In [7]:
top_cities = list(top_10.city.unique())
hex_top = list(near_mun.loc[near_mun.city.isin(top_cities)].hex_id_8.unique())

In [8]:
year_dict = {'10':'nearshoring_denue_10_hex',
            '15':'nearshoring_denue_15_hex',
            '20':'nearshoring_denue_20_hex'}

In [9]:
for y in year_dict.keys():
    table = year_dict[y]
    query = f"SELECT * FROM {schema}.{table} WHERE \"hex_id\" IN {str(tuple(hex_top))} "
    near_tmp = aup.gdf_from_query(query, geometry_col='geometry')
    near_tmp = near_tmp.drop(columns=['geometry'])
    near_mrg = gdf_mun.merge(near_tmp, right_on=f'hex_id',
                         left_on='hex_id_8')
    top_tmp = near_mrg.groupby('city').sum(numeric_only=True)
    top_tmp = top_tmp.add_prefix(f'{y}_')
    top_tmp = top_tmp.reset_index()
    top_10 = top_10.merge(top_tmp, on='city')

In [10]:
top_10 = top_10[top_10.columns.drop(list(top_10.filter(regex='pct')))]
top_10 = top_10[top_10.columns.drop(list(top_10.filter(regex='_res')))]
top_10

,city,23_nearshoring_industry,23_per_ocu_num_near_sum,23_denue,23_per_ocu_num_tot_sum,10_nearshoring_industry,10_per_ocu_num_near_sum,10_denue,10_per_ocu_num_tot_sum,15_nearshoring_industry,15_per_ocu_num_near_sum,15_denue,15_per_ocu_num_tot_sum,20_nearshoring_industry,20_per_ocu_num_near_sum,20_denue,20_per_ocu_num_tot_sum
0,CDMX,1075,43929,20018,257146,977,44322,16880,253095,937,42626,16736,242750,1026,41661,19094,251571
1,Guadalajara,546,35854,9379,176147,293,14408,6002,108994,358,20650,7005,122010,470,28701,8577,160367
2,ZMVM,536,30237,14081,208865,267,17603,7115,142448,276,18730,8146,148382,475,25229,13239,185437
3,Monterrey,473,49819,5970,204899,200,16314,3438,84768,262,29390,3783,115690,417,43572,5324,181984
4,Tijuana,301,37714,2086,119462,139,19722,977,55245,219,28484,1312,83435,262,33952,1790,103819
5,Queretaro,269,33968,2356,99178,130,16267,1280,49593,147,21568,1471,67511,237,28437,2125,81324
6,Puebla,237,18292,3976,66202,143,10612,2129,33132,162,12133,2535,40064,224,16608,3671,58830
7,Juarez,233,51744,1115,97497,163,35971,782,64601,164,38828,784,66690,214,45158,980,79125
8,Toluca,228,17442,3073,70483,99,5924,1418,31166,138,12204,1566,43648,214,14249,2944,59175
9,Saltillo,163,29441,1202,59764,62,9850,669,22808,84,13832,677,28416,141,24649,1066,47651


## Anual comparison

In [15]:
# nearshoring industry comparison
top_10['ns_23_10'] = top_10['23_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_23_10'] = top_10['ns_23_10'] / top_10['10_nearshoring_industry']
top_10['ns_23_20'] = top_10['23_nearshoring_industry'] - top_10['20_nearshoring_industry']
top_10['pct_ns_23_20'] = top_10['ns_23_20'] / top_10['20_nearshoring_industry']
top_10['ns_20_15'] = top_10['20_nearshoring_industry'] - top_10['15_nearshoring_industry']
top_10['pct_ns_20_15'] = top_10['ns_20_15'] / top_10['15_nearshoring_industry']
top_10['ns_20_10'] = top_10['20_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_20_10'] = top_10['ns_20_10'] / top_10['10_nearshoring_industry']
top_10['ns_15_10'] = top_10['15_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_15_10'] = top_10['ns_15_10'] / top_10['10_nearshoring_industry']
# nearshoring work comparison
top_10['ns_po_23_10'] = top_10['23_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_23_10'] = top_10['ns_po_23_10'] / top_10['10_per_ocu_num_near_sum']
top_10['ns_po_23_20'] = top_10['23_per_ocu_num_near_sum'] - top_10['20_per_ocu_num_near_sum']
top_10['pct_ns_po_23_20'] = top_10['ns_po_23_20'] / top_10['20_per_ocu_num_near_sum']
top_10['ns_po_20_15'] = top_10['20_per_ocu_num_near_sum'] - top_10['15_per_ocu_num_near_sum']
top_10['pct_ns_po_20_15'] = top_10['ns_po_20_15'] / top_10['15_per_ocu_num_near_sum']
top_10['ns_po_20_10'] = top_10['20_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_20_10'] = top_10['ns_po_20_10'] / top_10['10_per_ocu_num_near_sum']
top_10['ns_po_15_10'] = top_10['15_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_15_10'] = top_10['ns_po_15_10'] / top_10['10_per_ocu_num_near_sum']

In [ ]:
# denue industry comparison
top_10['ns_23_10'] = top_10['23_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_23_10'] = top_10['ns_23_10'] / top_10['10_nearshoring_industry']
top_10['ns_23_20'] = top_10['23_nearshoring_industry'] - top_10['20_nearshoring_industry']
top_10['pct_ns_23_20'] = top_10['ns_23_20'] / top_10['20_nearshoring_industry']
top_10['ns_20_15'] = top_10['20_nearshoring_industry'] - top_10['15_nearshoring_industry']
top_10['pct_ns_20_15'] = top_10['ns_20_15'] / top_10['15_nearshoring_industry']
top_10['ns_20_10'] = top_10['20_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_20_10'] = top_10['ns_20_10'] / top_10['10_nearshoring_industry']
top_10['ns_15_10'] = top_10['15_nearshoring_industry'] - top_10['10_nearshoring_industry']
top_10['pct_ns_15_10'] = top_10['ns_15_10'] / top_10['10_nearshoring_industry']
# denue work comparison
top_10['ns_po_23_10'] = top_10['23_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_23_10'] = top_10['ns_po_23_10'] / top_10['10_per_ocu_num_near_sum']
top_10['ns_po_23_20'] = top_10['23_per_ocu_num_near_sum'] - top_10['20_per_ocu_num_near_sum']
top_10['pct_ns_po_23_20'] = top_10['ns_po_23_20'] / top_10['20_per_ocu_num_near_sum']
top_10['ns_po_20_15'] = top_10['20_per_ocu_num_near_sum'] - top_10['15_per_ocu_num_near_sum']
top_10['pct_ns_po_20_15'] = top_10['ns_po_20_15'] / top_10['15_per_ocu_num_near_sum']
top_10['ns_po_20_10'] = top_10['20_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_20_10'] = top_10['ns_po_20_10'] / top_10['10_per_ocu_num_near_sum']
top_10['ns_po_15_10'] = top_10['15_per_ocu_num_near_sum'] - top_10['10_per_ocu_num_near_sum']
top_10['pct_ns_po_15_10'] = top_10['ns_po_15_10'] / top_10['10_per_ocu_num_near_sum']

In [17]:
# denue industry comparison
top_10['denue_23_10'] = top_10['23_denue'] - top_10['10_denue']
top_10['pct_denue_23_10'] = top_10['denue_23_10'] / top_10['10_denue']
top_10['denue_23_20'] = top_10['23_denue'] - top_10['20_denue']
top_10['pct_denue_23_20'] = top_10['denue_23_20'] / top_10['20_denue']
top_10['denue_20_15'] = top_10['20_denue'] - top_10['15_denue']
top_10['pct_denue_20_15'] = top_10['denue_20_15'] / top_10['15_denue']
top_10['denue_20_10'] = top_10['20_denue'] - top_10['10_denue']
top_10['pct_denue_20_10'] = top_10['denue_20_10'] / top_10['10_denue']
top_10['denue_15_10'] = top_10['15_denue'] - top_10['10_denue']
top_10['pct_denue_15_10'] = top_10['denue_15_10'] / top_10['10_denue']

# denue work comparison
top_10['denue_po_23_10'] = top_10['23_per_ocu_num_tot_sum'] - top_10['10_per_ocu_num_tot_sum']
top_10['pct_denue_po_23_10'] = top_10['denue_po_23_10'] / top_10['10_per_ocu_num_tot_sum']
top_10['denue_po_23_20'] = top_10['23_per_ocu_num_tot_sum'] - top_10['20_per_ocu_num_tot_sum']
top_10['pct_denue_po_23_20'] = top_10['denue_po_23_20'] / top_10['20_per_ocu_num_tot_sum']
top_10['denue_po_20_15'] = top_10['20_per_ocu_num_tot_sum'] - top_10['15_per_ocu_num_tot_sum']
top_10['pct_denue_po_20_15'] = top_10['denue_po_20_15'] / top_10['15_per_ocu_num_tot_sum']
top_10['denue_po_20_10'] = top_10['20_per_ocu_num_tot_sum'] - top_10['10_per_ocu_num_tot_sum']
top_10['pct_denue_po_20_10'] = top_10['denue_po_20_10'] / top_10['10_per_ocu_num_tot_sum']
top_10['denue_po_15_10'] = top_10['15_per_ocu_num_tot_sum'] - top_10['10_per_ocu_num_tot_sum']
top_10['pct_denue_po_15_10'] = top_10['denue_po_15_10'] / top_10['10_per_ocu_num_tot_sum']

In [18]:
top_10

,city,23_nearshoring_industry,23_per_ocu_num_near_sum,23_denue,23_per_ocu_num_tot_sum,10_nearshoring_industry,10_per_ocu_num_near_sum,10_denue,10_per_ocu_num_tot_sum,15_nearshoring_industry,...,denue_po_23_10,pct_denue_po_23_10,denue_po_23_20,pct_denue_po_23_20,denue_po_20_15,pct_denue_po_20_15,denue_po_20_10,pct_denue_po_20_10,denue_po_15_10,pct_denue_po_15_10
0,CDMX,1075,43929,20018,257146,977,44322,16880,253095,937,...,4051,0.016006,5575,0.022161,8821,0.036338,-1524,-0.006021,-10345,-0.040874
1,Guadalajara,546,35854,9379,176147,293,14408,6002,108994,358,...,67153,0.616116,15780,0.098399,38357,0.314376,51373,0.471338,13016,0.119419
2,ZMVM,536,30237,14081,208865,267,17603,7115,142448,276,...,66417,0.466254,23428,0.126339,37055,0.249727,42989,0.301787,5934,0.041657
3,Monterrey,473,49819,5970,204899,200,16314,3438,84768,262,...,120131,1.417174,22915,0.125918,66294,0.573031,97216,1.146848,30922,0.364784
4,Tijuana,301,37714,2086,119462,139,19722,977,55245,219,...,64217,1.162404,15643,0.150676,20384,0.244310,48574,0.879247,28190,0.510272
5,Queretaro,269,33968,2356,99178,130,16267,1280,49593,147,...,49585,0.999839,17854,0.219542,13813,0.204604,31731,0.639828,17918,0.361301
6,Puebla,237,18292,3976,66202,143,10612,2129,33132,162,...,33070,0.998129,7372,0.125310,18766,0.468401,25698,0.775625,6932,0.209224
7,Juarez,233,51744,1115,97497,163,35971,782,64601,164,...,32896,0.509218,18372,0.232190,12435,0.186460,14524,0.224826,2089,0.032337
8,Toluca,228,17442,3073,70483,99,5924,1418,31166,138,...,39317,1.261535,11308,0.191094,15527,0.355732,28009,0.898704,12482,0.400501
9,Saltillo,163,29441,1202,59764,62,9850,669,22808,84,...,36956,1.620309,12113,0.254202,19235,0.676907,24843,1.089223,5608,0.245879


In [20]:
top_10.to_csv('../../../data/processed/nearshoring/top_10_cities_analysis.csv')